In [0]:
%sql
select * from music.silver_album as album
join music.silver_artist as artist on album.artist_id = artist.id
limit 10

In [0]:
%sql
select * from music.silver_song as song
join music.silver_album as album on song.album_id = album.id
join music.silver_artist as artist on album.artist_id = artist.id
limit 50

In [0]:
%sql
select * from music.silver_album_club_member

In [0]:
%sql
select * from music.silver_member_rating

In [0]:
%sql
--select count(*) from music.bronze_member_rating;
--780
select count(*) from music.silver_member_rating;
--767

In [0]:
%sql
select member_id, count(*) from music.silver_member_rating
group by member_id

In [0]:
bronze_df = spark.table("workspace.music.bronze_member_rating")
silver_df = spark.table("workspace.music.silver_member_rating")
silver_member_df = spark.table("workspace.music.silver_album_club_member")

# Join silver_df with silver_member_df on member names
silver_df = silver_df.join(
    silver_member_df,
    silver_df["member_id"] == silver_member_df["id"],
    "inner"
)
silver_df = silver_df.withColumnRenamed("short_name", "member_name")
bronze_df = bronze_df.withColumnRenamed("member", "member_name")

missing_rows_df = bronze_df.join(
    silver_df,
    on=["week_number", "meeting_date", "member_name"],
    how="left_anti"
)

In [0]:
from pyspark.sql.functions import count

missing_rows_grouped_df = missing_rows_df.groupBy("album_name").agg(count("*").alias("count"))
display(missing_rows_grouped_df)

In [0]:
silver_album_df = spark.table("workspace.music.silver_album")

# Join silver_df with silver_album_df on album id
silver_df_w_albums = silver_df.join(
    silver_album_df,
    silver_df["album_id"] == silver_album_df["id"],
    "inner"
).drop(silver_album_df["id"])

# display(silver_df_w_albums)
display(silver_df_w_albums.filter(col("name").ilike("%blond%")))

In [0]:
%sql
select * from music.bronze_album al where al.album_name like '%Born%Die%'

In [0]:
%sql
select * from music.bronze_member_rating al where al.album_name like '%Born%Die%'

In [0]:
%sql
update music.bronze_member_rating
set album_name = 'Born to Die' where album_name = 'Born To Die'